# Marketing Assistant

Copyright 2025, Denis Rothman

**Goal:** This notebook serves as the practical runtime for the **Strategic Marketing Engine**, as architected in Chapter 9. It demonstrates how the generic, multi-domain "Glass Box" Context Engine can be seamlessly repurposed to solve a variety of real-world marketing challenges without any changes to its core code.

This notebook will:
* Connect to the Pinecone knowledge base populated by the `Data_Ingestion_Marketing.ipynb` script.
* Use the three generic Control Deck templates to interact with the engine.
* Execute seven distinct marketing use cases, from enforcing brand voice to drafting a complete email nurture sequence.




# I. Inititalization

## GitHub

In [1]:
# --- Downloading the utilities and the agents ---

# The !curl command is a simple and effective way to download raw files from a public GitHub repo.
# The -L flag ensures that it follows any redirects.

# print("Downloading helper files from public repository...")
# !curl -L https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/utils.py --output utils.py
# !curl -L https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/helpers.py --output helpers.py
# !curl -L https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/agents.py --output agents.py
# !curl -L https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/registry.py.py --output registry.py.py
# !curl -L https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/engine.py --output engine.py

# print("✅ Files downloaded successfully!")

In [2]:
import requests
import os
from google.colab import userdata

def download_private_github_file(filename, destination_path="."):
    """
    Downloads a file from a private GitHub repository using a token
    stored in Colab Secrets.

    Requires a secret named 'GITHUB_TOKEN' to be set in the Colab UI.
    """
    # --- Configuration: Replace with your repository details ---
    owner = "Denis2054"
    repo = "Context-Engineering"
    branch = "main"
    # ---------------------------------------------------------

    try:
        # Securely fetch the GitHub token from Colab Secrets
        token = userdata.get('GITHUB_TOKEN')
        if not token:
            raise userdata.SecretNotFoundError("Secret 'GITHUB_TOKEN' not found.")
    except userdata.SecretNotFoundError:
        print("🛑 Error: Secret 'GITHUB_TOKEN' not found.")
        print("Please add your GitHub Personal Access Token to the Colab Secrets Manager.")
        return
    except Exception as e:
        print(f"An error occurred while accessing secrets: {e}")
        return

    # Construct the GitHub API URL for the file
    url = f"https://api.github.com/repos/{owner}/{repo}/contents/{filename}?ref={branch}"

    # Prepare headers for authentication and to request the raw file content
    headers = {
        "Authorization": f"Bearer {token}",
        "Accept": "application/vnd.github.v3.raw"
    }

    try:
        # Make the request to the GitHub API
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # This will raise an HTTPError for bad responses (4xx or 5xx)

        # Determine the local filename
        local_filename = os.path.join(destination_path, os.path.basename(filename))

        # Save the file content locally
        with open(local_filename, "wb") as f:
            f.write(response.content)

        print(f"✅ Successfully downloaded '{filename}' to '{local_filename}'")

    except requests.exceptions.HTTPError as e:
        print(f"🛑 Error downloading '{filename}': {e}")
        if e.response.status_code == 404:
            print("   Please check that the owner, repo, file path, and branch are correct.")
        elif e.response.status_code == 401:
            print("   Authentication failed. Please check if your GITHUB_TOKEN is valid and has access to the repo.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


# --- Example Usage: Download your utility files ---
# Ensure you have set 'GITHUB_TOKEN' in your Colab Secrets before running this.

download_private_github_file("commons/ch8/utils.py")
download_private_github_file("commons/ch8/helpers.py")
download_private_github_file("commons/ch8/agents.py")
download_private_github_file("commons/ch8/registry.py")
download_private_github_file("commons/ch8/engine.py")

✅ Successfully downloaded 'commons/ch8/utils.py' to './utils.py'
✅ Successfully downloaded 'commons/ch8/helpers.py' to './helpers.py'
✅ Successfully downloaded 'commons/ch8/agents.py' to './agents.py'
✅ Successfully downloaded 'commons/ch8/registry.py' to './registry.py'
✅ Successfully downloaded 'commons/ch8/engine.py' to './engine.py'


## Installation and client setup

In [3]:
#Installation and Client Setup

# Import the setup functions from your new utility file
import utils

# Run the installation
utils.install_dependencies()

# Initialize the OpenAI and Pinecone clients
client, pc = utils.initialize_clients()

🚀 Installing required packages...
✅ All packages installed successfully.

🔑 Initializing API clients...
   - OpenAI client initialized.
   - Pinecone client initialized.
✅ Clients initialized successfully.


## Context Engine library Import

In [4]:
# 1. Import the hardened helper functions (LLM, Embeddings, Pinecone)
import helpers

# 2. Import the specialist agent functions (Librarian, Researcher, Writer)
import agents

# 3. Import the AGENT_TOOLKIT object that knows about all the agents
from registry import AGENT_TOOLKIT

# 4. Import the main context_engine function that orchestrates the entire process
from engine import context_engine

## Engine Room

In [5]:
# === ENGINE ROOM: The Main Execution Function ===
# This function contains all the logic to run the engine.
# We define it here so our final cell can be very simple.

import logging
import pprint
from IPython.display import display, Markdown

# In Legal_Compliance_Assistant.ipynb (The "Engine Room" cell)

def execute_and_display(goal, config, client, pc, moderation_active=False):
    """
    Runs the context engine, now with an optional, two-stage moderation check.
    """
    # --- PRE-FLIGHT MODERATION CHECK (on user input) ---
    if moderation_active:
        print("--- [Safety Guardrail] Performing Pre-Flight Moderation Check on Goal ---")
        moderation_report = helpers.helper_moderate_content(text_to_moderate=goal, client=client)

        print("Moderation Report:")
        pprint.pprint(moderation_report)

        if moderation_report["flagged"]:
            print("\n🛑 Goal failed pre-flight moderation. Execution halted.")
            return # Halt execution before calling the engine

    logging.info(f"******** Starting Engine for Goal: '{goal}' **********\\n")

    # 1. Run the Context Engine using the provided configuration
    result, trace = context_engine(
        goal,
        client=client,
        pc=pc,
        **config
    )

    # --- POST-FLIGHT MODERATION CHECK (on AI output) ---
    if result and moderation_active:
        print("\n--- [Safety Guardrail] Performing Post-Flight Moderation Check on Output ---")
        moderation_report = helpers.helper_moderate_content(text_to_moderate=result, client=client)

        print("Moderation Report:")
        pprint.pprint(moderation_report)

        if moderation_report["flagged"]:
            print("\n🛑 Generated output failed post-flight moderation and will be redacted.")
            result = "[Content flagged as potentially harmful by moderation policy and has been redacted.]"

    # 2. Display the Final Result
    print("\n--- FINAL OUTPUT ---")
    if result:
        display(Markdown(result))
    else:
        print(f"The engine failed to produce a result. Status: {trace.status}")

    # 3. Display the Technical Trace
    print("\\n\\n--- TECHNICAL TRACE (for the tech reader) ---")
    if trace:
        print(f"Trace Status: {trace.status}")
        print(f"Total Duration: {trace.duration:.2f} seconds")
        print("Execution Steps:")
        # --- THIS LINE IS TO CREATE THE 'pp' OBJECT ---
        pp = pprint.PrettyPrinter(indent=2)

        pp.pprint(trace.steps)


## Control Deck configuration

In [6]:
# 1. Define all configuration variables for this run in a dictionary
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5",
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

#III.CONTROL DECKS

=== CONTROL DECK: Define Goal and Run Engine ===
This is the main interactive cell.
1. Change the 'goal' variable to your desired task.
2. Run this cell.


In [7]:
#@title CONTROL DECK: Moderation
# 1. Define a simple, safe goal to test the moderation workflow.
goal = "Summarize the key points of the Non-Disclosure Agreement."

# 2. Define the standard configuration.
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5",
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

# 3. Call the execution function with moderation explicitly activated.
execute_and_display(goal, config, client, pc, moderation_active=True)

--- [Safety Guardrail] Performing Pre-Flight Moderation Check on Goal ---
Moderation Report:
{'categories': {'harassment': False,
                'harassment/threatening': False,
                'harassment_threatening': False,
                'hate': False,
                'hate/threatening': False,
                'hate_threatening': False,
                'illicit': False,
                'illicit/violent': False,
                'illicit_violent': False,
                'self-harm': False,
                'self-harm/instructions': False,
                'self-harm/intent': False,
                'self_harm': False,
                'self_harm_instructions': False,
                'self_harm_intent': False,
                'sexual': False,
                'sexual/minors': False,
                'sexual_minors': False,
                'violence': False,
                'violence/graphic': False,
                'violence_graphic': False},
 'flagged': False,
 'scores': {'harassment': 4

ERROR:root:--- Executor: FATAL ERROR --- Execution failed at step 2 (Summarizer): Dependency Error: Reference NDA_DOCUMENT_TEXT not found in state.



--- FINAL OUTPUT ---
The engine failed to produce a result. Status: Failed at Step 2
\n\n--- TECHNICAL TRACE (for the tech reader) ---
Trace Status: Failed at Step 2
Total Duration: 36.09 seconds
Execution Steps:
[ { 'agent': 'Librarian',
    'output': { 'blueprint_json': '{"scene_goal": "Explain the mechanism or '
                                  'findings clearly and concisely.", '
                                  '"style_guide": "Maintain an objective and '
                                  'formal tone. Use precise terminology. '
                                  'Prioritize factual accuracy and clarity '
                                  'over narrative flair.", "structure": '
                                  '["Definition", "Function/Operation", "Key '
                                  'Findings/Impact"], "instruction": "Organize '
                                  'the provided facts into the defined '
                                  'structure, adhering to the style_guide

In [9]:
#@title CONTROL DECK 1: Product Marketing Copy Generation(Use Case 2)

# 1. Define the Goal: A research query that asks for a creative output.
goal = "Using the official product spec sheet, write a short marketing description for the new QuantumDrive Q-1. The description should be confident, aspirational, and focus on the benefits for creative professionals. Please cite your sources."


# 1. Define the Goal: A research query that requires a verifiable, cited answer.
#    - DOMAIN: Any knowledge-intensive field (e.g., legal, medical, financial).
#    - KEY CAPABILITY: Tests the high-fidelity `Researcher` agent and its ability
#      to retrieve text with `source` metadata and generate citations.
# goal = "[INSERT YOUR HIGH-FIDELITY RESEARCH GOAL HERE]"

# 1. Define the Goal: A research query focused on strategic analysis.
goal = "Analyze the ChronoTech press release and summarize their core product messaging and value proposition. Please cite your sources."


# 2. Use the standard configuration
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5", # or your preferred model
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

# 3. Call the execution function
execute_and_display(goal, config, client, pc,moderation_active=False)


--- FINAL OUTPUT ---


Definition
ChronoTech positions the Chrono SSD Pro as a creator-focused, flagship NVMe SSD built on a proven PCIe Gen4 platform. It targets users who prioritize maximum read/write performance for demanding creative workloads. The 4TB capacity is competitively priced and available for immediate purchase.

Function/Operation
The drive delivers up to 7,300 MB/s sequential read speeds, aiming to reduce wait times and eliminate workflow bottlenecks. Its PCIe Gen4 foundation is intended to provide high, predictable performance on supported systems. Immediate availability enables rapid deployment without upgrade delays.

Key Findings/Impact
- Prioritizing raw speed helps maintain smooth project throughput, ensuring technology does not impede creative processes.
- Competitive 4TB pricing offers pro-level storage at an accessible cost for budget-conscious creators.
- As a flagship model, it is suited to high-intensity creative tasks where fast iteration and data movement are critical.

\n\n--- TECHNICAL TRACE (for the tech reader) ---
Trace Status: Success
Total Duration: 50.28 seconds
Execution Steps:
[ { 'agent': 'Researcher',
    'output': { 'answer_with_sources': 'Official ChronoTech press release '
                                       '(latest available)\n'
                                       '\n'
                                       'Canonical URL(s): Not provided in the '
                                       'supplied sources.\n'
                                       '\n'
                                       'Press release text:\n'
                                       'FOR IMMEDIATE RELEASE [1]\n'
                                       '\n'
                                       'ChronoTech Unveils the Chrono SSD Pro: '
                                       'Speed for the Modern Creator [1]\n'
                                       '\n'
                                       'CUPERTINO, CA – ChronoTech today '
                                   

In [10]:
#@title CONTROL DECK 1: Competitor Analysis (Use Case 3)


# 1. Define the Goal: A research query that requires a verifiable, cited answer.
#    - DOMAIN: Any knowledge-intensive field (e.g., legal, medical, financial).
#    - KEY CAPABILITY: Tests the high-fidelity `Researcher` agent and its ability
#      to retrieve text with `source` metadata and generate citations.
# goal = "[INSERT YOUR HIGH-FIDELITY RESEARCH GOAL HERE]"

## 1. Define the Goal: A research query that asks for a creative output.
goal = "Using the official product spec sheet, write a short marketing description for the new QuantumDrive Q-1. The description should be confident, aspirational, and focus on the benefits for creative professionals. Please cite your sources."



# 2. Use the standard configuration
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5", # or your preferred model
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

# 3. Call the execution function
execute_and_display(goal, config, client, pc,moderation_active=False)


--- FINAL OUTPUT ---


Definition
QuantumDrive Q-1 is a PCIe Gen 5 NVMe 2.0 solid-state drive designed for creative professionals, available in 2TB, 4TB, and 8TB capacities.

Function/Operation
- Interface: NVMe 2.0 over PCIe Gen 5 delivers high bandwidth and low latency for next‑generation workstations.
- Performance: Up to 7,500 MB/s sequential read and 7,000 MB/s sequential write, supporting smooth multi‑stream 4K/8K timeline playback, rapid renders/exports, and high‑speed scratch/cache workflows.
- Thermals: An integrated graphene heat spreader helps prevent thermal throttling under sustained loads.
- Security: AES‑256 hardware encryption with the included DataWeaver backup/encryption suite for on‑set and secured workflows.

Key Findings/Impact
- Capacity headroom supports large media libraries, proxy generation, and project caches.
- Sustained performance and reliability are reinforced by thermal management and a 5‑year limited warranty.
- Endurance: The 4TB model is rated at 3,000 TBW (terabytes written).
- The PCIe Gen 5/NVMe 2.0 stack improves responsiveness and throughput for demanding creative workloads while maintaining low latency.

\n\n--- TECHNICAL TRACE (for the tech reader) ---
Trace Status: Success
Total Duration: 53.52 seconds
Execution Steps:
[ { 'agent': 'Librarian',
    'output': { 'blueprint_json': '{"scene_goal": "Explain the mechanism or '
                                  'findings clearly and concisely.", '
                                  '"style_guide": "Maintain an objective and '
                                  'formal tone. Use precise terminology. '
                                  'Prioritize factual accuracy and clarity '
                                  'over narrative flair.", "structure": '
                                  '["Definition", "Function/Operation", "Key '
                                  'Findings/Impact"], "instruction": "Organize '
                                  'the provided facts into the defined '
                                  'structure, adhering to the style_guide."}'},
    'planned_input': { 'intent_query': 'Blueprint for a short marketing '
              

In [11]:
#@title CONTROL DECK TEMPLATE 3: Writing a brand pitch recommendation(Use case 4)

# 1. Define the Goal: A creative or factual task that is deliberately
#    outside the scope of the documents in the knowledge base.
#    - DOMAIN: Universal test applicable to any curated knowledge base.
#    - KEY CAPABILITY: Tests the `Researcher` agent's ability to report a
#      negative finding and the `Writer` agent's ability to handle it gracefully,
#      preventing hallucination.
# goal = "[INSERT YOUR OUT-OF-SCOPE GOAL HERE]"


# =CONTROL DECK 3: A persuasive pitch ===
goal = "Write a persuasive pitch on our brand tone and voice guide"


# 2. Use the same configuration dictionary
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5", # or your preferred model
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

# 3. Call the execution function
execute_and_display(goal, config, client, pc,moderation_active=False)


--- FINAL OUTPUT ---


Quick take: adopt the Innovate Forward voice guide (Clarity, Confidence, Aspiration) and you’ll align teams, speed up content, and make results trackable right away.

Why it matters
- One voice, one story: “End the Wait,” “Built for Pros,” “The Ultimate Upgrade.” Same headlines and CTAs across social and email = better recall and trust.
- Unified funnel: Social and nurture follow the same arc—Problem → Solution → Proof → Pre-order—so every touch pushes to one trackable action.
- Measurable by design: Single CTA with a trackable link lets you see exactly what converts.

Conversion and CAC boost
- Nurture flow that builds intent: 
  - Email 1: call out the pain (slow storage).
  - Email 2: present QuantumDrive—“End the Wait.”
  - Email 3: proof + 5-year warranty, then the pre-order ask.
- Short, active, clear copy lowers friction → more clicks, more pre-orders.
- Better conversion + fewer rewrites = lower blended CAC and lower content costs.

Faster production
- Less back-and-forth: guardrails (no hedging, no overpromises, no negative competitor talk) and a checklist mean “write once to standard.”
- Templates: social brief and 3-email outline become reusable building blocks.

Onboarding made easy
- One source of truth: three principles, do/don’t list, and live examples.
- Reviewer checklist: 
  - Clarity: simple words, short sentences, scannable.
  - Confidence: active voice, direct benefits, no hedging.
  - Aspiration: future-focused, tied to user goals.
  - Compliance: no slang, no unprovable claims, no negative comparisons.

Avoid the risks
- Inconsistent tone = confusion, weaker trust, slower approvals, and lost recall if CTAs/headlines wander from “End the Wait.”

Rollout, adoption, governance
- Publish the guide + one-page checklist; turn Q1 assets into evergreen templates; stock a library of approved headlines/bullets/CTAs.
- Require the checklist on every draft; one CTA per asset with the campaign link standard.
- Assign an owner and lightweight review squad; refresh examples quarterly; keep a shared repo; add a “voice compliance” field in requests.

What to track
- Funnel: social clicks and pre-order rate; Email 2 CTR; Email 3 pre-order rate; step progression; total pre-orders.
- Efficiency: time to first draft/approval; revision cycles; first-pass checklist pass rate.
- Brand consistency: internal compliance score; coverage of key messages; warranty mention rate.
- CAC: cost per asset, per acquired customer, and per pre-order (via trackable links).

Time-to-value and ROI
- Immediate: apply the checklist to current Q1 social and the 3-email sequence; standardize CTAs now.
- Near term: track clicks and pre-orders to quantify lift.
- ROI lens: more pre-orders from clearer messaging; lower costs from fewer revisions and reusable templates; governance keeps the gains.

Note: The plan is qualitative—set baselines now and instrument the KPIs to build your own credible performance data.

\n\n--- TECHNICAL TRACE (for the tech reader) ---
Trace Status: Success
Total Duration: 63.21 seconds
Execution Steps:
[ { 'agent': 'Librarian',
    'output': { 'blueprint_json': '{"scene_goal": "Summarize information '
                                  'quickly and casually.", "style_guide": "Use '
                                  'informal language. Keep it brief and '
                                  'engaging. Imagine explaining it to a '
                                  'friend.", "instruction": "Summarize the '
                                  'provided facts using the casual style '
                                  'guide."}'},
    'planned_input': { 'intent_query': 'Persuasive internal pitch memo '
                                       'advocating adoption and consistent use '
                                       'of a brand tone and voice guide; '
                                       'structure: attention-grabbing hook, '
                                       'curre